## Import Modules

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from os import listdir
from os.path import isfile, join
import time
import math
from IPython.display import clear_output
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import torchvision.transforms as T
#from torchsampler import ImbalancedDatasetSampler
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print('cuda available: '+ str(torch.cuda.is_available()))
#from skimage.morphology import disk, binary_dilation

from models_avm import NestedUNet
from loss_fun_avm import compute_per_channel_dice, DiceLoss, FocalLoss
from tra_val_avm import train, validation
from data_loader_avm import Dataset

cuda available: True


## Data Path

In [2]:
path_tra1 = '/work/samhong833/Data_AVM/forUNetpp/1_tra/1'
list_tra1 = os.listdir(path_tra1)
for i in range(len(list_tra1)):
    list_tra1[i] = path_tra1+'/'+list_tra1[i]
    
list_tra = list_tra1
    
path_val1 = '/work/samhong833/Data_AVM/forUNetpp/2_val/1'
list_val1 = os.listdir(path_val1)
for i in range(len(list_val1)):
    list_val1[i] = path_val1+'/'+list_val1[i]
    
list_val = list_val1

path_ts1 = '/work/samhong833/Data_AVM/forUNetpp/3_ts/1'
list_ts1 = os.listdir(path_ts1)
for i in range(len(list_ts1)):
    list_ts1[i] = path_ts1+'/'+list_ts1[i]
    
list_ts = list_ts1

path_tra_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/1_tra'
path_val_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/2_val'
path_ts_lab_txt = '/work/samhong833/Data_AVM/forYOLOv5/labels/3_ts'

## Data Loader

In [3]:
train_data = Dataset(list_tra,path_tra_lab_txt,rand_dilate=False)
train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size = 12,
    shuffle = True,   
)

val_data = Dataset(list_val,path_val_lab_txt,rand_dilate=False)
val_loader = torch.utils.data.DataLoader(
    dataset = val_data,           
    batch_size = 4,                 
    shuffle = False,              
)

## Initialize Model and Optimization Parameters

In [4]:
# call model cuda for gpu
model = NestedUNet().cuda()

# define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.95)

# loss function
kwargs = {"alpha": 0.5, "gamma": 3, "reduction": 'mean'}
criterion_FL = FocalLoss(**kwargs)
criterion_DICE = DiceLoss()
loss = [criterion_FL,criterion_DICE]

## Start to Train

In [ ]:
# Create Directory
path = '/work/samhong833/Models/Seg_nodia'

if os.path.isdir(path)==False:
    os.mkdir(path)
path = os.path.join(path,"train")
if os.path.isdir(path)==False:
    os.mkdir(path)       
filenum = glob.glob(path + "/exp*")
path = path + "/exp" + str(len(filenum)+1)
os.mkdir(path)

# Train the Model
epochs = 500 # The number of epochs

valloss = 0
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(path=path,
          model=model,
          loss=loss,
          optimizer=optimizer,
          dataloader=train_loader,
          epoch=epoch,
          scheduler=scheduler)
    print('-' * 89)
    vallossnew = validation(path=path,
          model=model,
          loss=loss,
          dataloader=val_loader,
          epoch=epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s'.format(epoch, (time.time() - epoch_start_time)))
    print('-' * 89)
    
    scheduler.step()
    
    f2 = open(path + '/model_info.txt', 'a')
    if vallossnew<valloss or epoch ==1: 
        fname = path + '/best_val'  + '.tar'
        torch.save(model.state_dict(), fname)
        valloss = vallossnew
        f2.write('| best_val | epoch {:3d}| '.format(epoch)+'\r\n')        
    f2.close()
    

| epoch   1 |   100/  290 batches | lr 0.0001000 | ms/batch 122761.21 | loss_fl 0.003013 | loss_dice  0.12 | 
| epoch   1 |   200/  290 batches | lr 0.0001000 | ms/batch 119697.22 | loss_fl 0.001681 | loss_dice  0.10 | 
| epoch   1 |   290/  290 batches | lr 0.0001000 | ms/batch 107681.68 | loss_fl 0.001248 | loss_dice  0.09 | 
-----------------------------------------------------------------------------------------
| epoch   1 |   100/  118 batches | | ms/batch 25328.56 | val_loss_fl 0.000377 | val_loss_dice  0.07 | 
| epoch   1 |   118/  118 batches | | ms/batch 4536.71 | val_loss_fl 0.000376 | val_loss_dice  0.07 | 
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 380.01s
-----------------------------------------------------------------------------------------
| epoch   2 |   100/  290 batches | lr 0.0001000 | ms/batch 118617.91 | loss_fl 0.000266 | loss_dice  0.07 | 
| epoch   2 |   200/  290 batches | lr 0.0001000

| epoch  11 |   100/  290 batches | lr 0.0000950 | ms/batch 124698.00 | loss_fl 0.000160 | loss_dice  0.05 | 
| epoch  11 |   200/  290 batches | lr 0.0000950 | ms/batch 126030.74 | loss_fl 0.000161 | loss_dice  0.05 | 
| epoch  11 |   290/  290 batches | lr 0.0000950 | ms/batch 113319.04 | loss_fl 0.000163 | loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| epoch  11 |   100/  118 batches | | ms/batch 24617.76 | val_loss_fl 0.000275 | val_loss_dice  0.06 | 
| epoch  11 |   118/  118 batches | | ms/batch 4342.95 | val_loss_fl 0.000273 | val_loss_dice  0.06 | 
-----------------------------------------------------------------------------------------
| end of epoch  11 | time: 393.01s
-----------------------------------------------------------------------------------------
| epoch  12 |   100/  290 batches | lr 0.0000950 | ms/batch 125618.26 | loss_fl 0.000153 | loss_dice  0.05 | 
| epoch  12 |   200/  290 batches | lr 0.0000950

| epoch  21 |   100/  290 batches | lr 0.0000902 | ms/batch 124230.85 | loss_fl 0.000099 | loss_dice  0.04 | 
| epoch  21 |   200/  290 batches | lr 0.0000902 | ms/batch 124505.14 | loss_fl 0.000098 | loss_dice  0.04 | 
| epoch  21 |   290/  290 batches | lr 0.0000902 | ms/batch 111715.75 | loss_fl 0.000101 | loss_dice  0.04 | 
-----------------------------------------------------------------------------------------
| epoch  21 |   100/  118 batches | | ms/batch 21941.09 | val_loss_fl 0.000359 | val_loss_dice  0.05 | 
| epoch  21 |   118/  118 batches | | ms/batch 3751.03 | val_loss_fl 0.000352 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch  21 | time: 386.15s
-----------------------------------------------------------------------------------------
| epoch  22 |   100/  290 batches | lr 0.0000902 | ms/batch 124852.22 | loss_fl 0.000095 | loss_dice  0.04 | 
| epoch  22 |   200/  290 batches | lr 0.0000902

| epoch  31 |   100/  290 batches | lr 0.0000857 | ms/batch 121771.14 | loss_fl 0.000067 | loss_dice  0.03 | 
| epoch  31 |   200/  290 batches | lr 0.0000857 | ms/batch 124050.86 | loss_fl 0.000066 | loss_dice  0.03 | 
| epoch  31 |   290/  290 batches | lr 0.0000857 | ms/batch 111265.63 | loss_fl 0.000067 | loss_dice  0.03 | 
-----------------------------------------------------------------------------------------
| epoch  31 |   100/  118 batches | | ms/batch 25210.55 | val_loss_fl 0.000523 | val_loss_dice  0.05 | 
| epoch  31 |   118/  118 batches | | ms/batch 4502.20 | val_loss_fl 0.000519 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch  31 | time: 386.81s
-----------------------------------------------------------------------------------------
| epoch  32 |   100/  290 batches | lr 0.0000857 | ms/batch 124111.64 | loss_fl 0.000065 | loss_dice  0.03 | 
| epoch  32 |   200/  290 batches | lr 0.0000857

| epoch  41 |   100/  290 batches | lr 0.0000815 | ms/batch 120115.56 | loss_fl 0.000048 | loss_dice  0.02 | 
| epoch  41 |   200/  290 batches | lr 0.0000815 | ms/batch 118417.86 | loss_fl 0.000049 | loss_dice  0.02 | 
| epoch  41 |   290/  290 batches | lr 0.0000815 | ms/batch 107129.40 | loss_fl 0.000049 | loss_dice  0.02 | 
-----------------------------------------------------------------------------------------
| epoch  41 |   100/  118 batches | | ms/batch 24742.94 | val_loss_fl 0.000679 | val_loss_dice  0.05 | 
| epoch  41 |   118/  118 batches | | ms/batch 4290.98 | val_loss_fl 0.000677 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch  41 | time: 374.70s
-----------------------------------------------------------------------------------------
| epoch  42 |   100/  290 batches | lr 0.0000815 | ms/batch 115911.04 | loss_fl 0.000046 | loss_dice  0.02 | 
| epoch  42 |   200/  290 batches | lr 0.0000815

...

| epoch 141 |   100/  290 batches | lr 0.0000488 | ms/batch 120895.29 | loss_fl 0.000009 | loss_dice  0.00 | 
| epoch 141 |   200/  290 batches | lr 0.0000488 | ms/batch 120962.15 | loss_fl 0.000009 | loss_dice  0.00 | 
| epoch 141 |   290/  290 batches | lr 0.0000488 | ms/batch 109230.24 | loss_fl 0.000009 | loss_dice  0.00 | 
-----------------------------------------------------------------------------------------
| epoch 141 |   100/  118 batches | | ms/batch 22683.67 | val_loss_fl 0.002029 | val_loss_dice  0.05 | 
| epoch 141 |   118/  118 batches | | ms/batch 4247.04 | val_loss_fl 0.002031 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch 141 | time: 378.02s
-----------------------------------------------------------------------------------------
| epoch 142 |   100/  290 batches | lr 0.0000488 | ms/batch 121306.82 | loss_fl 0.000009 | loss_dice  0.00 | 
| epoch 142 |   200/  290 batches | lr 0.0000488

| epoch 151 |   100/  290 batches | lr 0.0000463 | ms/batch 119508.16 | loss_fl 0.000008 | loss_dice  0.00 | 
| epoch 151 |   200/  290 batches | lr 0.0000463 | ms/batch 119440.51 | loss_fl 0.000008 | loss_dice  0.00 | 
| epoch 151 |   290/  290 batches | lr 0.0000463 | ms/batch 108458.69 | loss_fl 0.000008 | loss_dice  0.00 | 
-----------------------------------------------------------------------------------------
| epoch 151 |   100/  118 batches | | ms/batch 22884.31 | val_loss_fl 0.002113 | val_loss_dice  0.05 | 
| epoch 151 |   118/  118 batches | | ms/batch 4218.95 | val_loss_fl 0.002115 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch 151 | time: 374.51s
-----------------------------------------------------------------------------------------
| epoch 152 |   100/  290 batches | lr 0.0000463 | ms/batch 120295.25 | loss_fl 0.000007 | loss_dice  0.00 | 
| epoch 152 |   200/  290 batches | lr 0.0000463

| epoch 161 |   100/  290 batches | lr 0.0000440 | ms/batch 119669.72 | loss_fl 0.000007 | loss_dice  0.00 | 
| epoch 161 |   200/  290 batches | lr 0.0000440 | ms/batch 119769.73 | loss_fl 0.000007 | loss_dice  0.00 | 
| epoch 161 |   290/  290 batches | lr 0.0000440 | ms/batch 107989.39 | loss_fl 0.000007 | loss_dice  0.00 | 
-----------------------------------------------------------------------------------------
| epoch 161 |   100/  118 batches | | ms/batch 18927.75 | val_loss_fl 0.002275 | val_loss_dice  0.05 | 
| epoch 161 |   118/  118 batches | | ms/batch 3975.40 | val_loss_fl 0.002284 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch 161 | time: 370.34s
-----------------------------------------------------------------------------------------
| epoch 162 |   100/  290 batches | lr 0.0000440 | ms/batch 119279.58 | loss_fl 0.000006 | loss_dice  0.00 | 
| epoch 162 |   200/  290 batches | lr 0.0000440

| epoch 171 |   100/  290 batches | lr 0.0000418 | ms/batch 111196.14 | loss_fl 0.000006 | loss_dice  0.00 | 
| epoch 171 |   200/  290 batches | lr 0.0000418 | ms/batch 111208.33 | loss_fl 0.000006 | loss_dice  0.00 | 
| epoch 171 |   290/  290 batches | lr 0.0000418 | ms/batch 98816.32 | loss_fl 0.000006 | loss_dice  0.00 | 
-----------------------------------------------------------------------------------------
| epoch 171 |   100/  118 batches | | ms/batch 19215.37 | val_loss_fl 0.002486 | val_loss_dice  0.05 | 
| epoch 171 |   118/  118 batches | | ms/batch 3504.77 | val_loss_fl 0.002486 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch 171 | time: 343.94s
-----------------------------------------------------------------------------------------
| epoch 172 |   100/  290 batches | lr 0.0000418 | ms/batch 112227.69 | loss_fl 0.000006 | loss_dice  0.00 | 
| epoch 172 |   200/  290 batches | lr 0.0000418 

| epoch 181 |   100/  290 batches | lr 0.0000397 | ms/batch 117386.50 | loss_fl 0.000006 | loss_dice  0.00 | 
| epoch 181 |   200/  290 batches | lr 0.0000397 | ms/batch 118034.51 | loss_fl 0.000005 | loss_dice  0.00 | 
| epoch 181 |   290/  290 batches | lr 0.0000397 | ms/batch 107083.61 | loss_fl 0.000005 | loss_dice  0.00 | 
-----------------------------------------------------------------------------------------
| epoch 181 |   100/  118 batches | | ms/batch 24471.21 | val_loss_fl 0.002521 | val_loss_dice  0.05 | 
| epoch 181 |   118/  118 batches | | ms/batch 4330.87 | val_loss_fl 0.002529 | val_loss_dice  0.05 | 
-----------------------------------------------------------------------------------------
| end of epoch 181 | time: 371.31s
-----------------------------------------------------------------------------------------
| epoch 182 |   100/  290 batches | lr 0.0000397 | ms/batch 118144.30 | loss_fl 0.000005 | loss_dice  0.00 | 
| epoch 182 |   200/  290 batches | lr 0.0000397

# Log ^